<a href="https://colab.research.google.com/github/rahiakela/data-science-research-and-practice/blob/main/data-extraction-works/03_exam_one_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reference**

https://www.geeksforgeeks.org/drop-rows-from-the-dataframe-based-on-certain-condition-applied-on-a-column/

https://datatofish.com/export-pandas-dataframe-json/

In [ ]:
!pip install -q tabula-py

     |████████████████████████████████| 12.0 MB 8.2 MB/s 


In [ ]:
import tabula
import pandas as pd

In [ ]:
# Read pdf into DataFrame
dfs = tabula.read_pdf("redacted1.pdf", lattice=True, relative_area=True, multiple_tables=True, pages="all")

# convert PDF into CSV
#tabula.convert_into("redacted1.pdf", "output.csv", output_format="csv")

Oct 24, 2022 6:26:53 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
def drop_unwanted_columns(df):
  wanted_columns = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']
  drop_columns = []
  for col in df.columns:
    if col in wanted_columns:
      drop_columns.append(col)
  data_df = df[df.columns.drop(drop_columns)]
  return data_df

In [ ]:
def update_df_columns(df):
  col_list = ["Encounter", "Specimen", "Service Date", "High Score", "Score", "Lab", "Doctor Name", "Location", "Phone"]

  curr_rename_dict = {}
  curr_new_dict = {}

  current_column = df.columns[0]

  for idx, col_name in enumerate(df.columns):
    curr_rename_dict[col_name] = col_list[idx]

  for idx, col_name in enumerate(df.columns):
    curr_new_dict[col_list[idx]] = col_name

  # renaming the columns
  df.rename(columns=curr_rename_dict, inplace = True)

  # adding a new row
  new_row_df = pd.DataFrame(curr_new_dict, index =[0])
  # Concatenate new_row with df
  updated_df = pd.concat([new_row_df, df[:]]).reset_index(drop = True)
  return updated_df

In [ ]:
def add_column_header(df, previous_column):
  col_list = ["Encounter", "Specimen", "Service Date", "High Score", "Score", "Lab", "Doctor Name", "Location", "Phone"]

  curr_rename_dict = {}
  curr_new_dict = {}

  curr_columns_values = list(df.columns)
  
  if previous_column == "Encounter":
    curr_columns_values.insert(1, "")

  for idx, col_name in enumerate(curr_columns_values):
    print(idx)
    if idx == 1:
      curr_rename_dict["Specimen"] = ""
    else:
      curr_rename_dict[col_name] = col_list[idx]

  for idx, col_name in enumerate(curr_columns_values):
    curr_new_dict[col_list[idx]] = col_name

  if previous_column == "Encounter":
    # renaming the columns
    df.rename(columns=curr_rename_dict, inplace = True)

  # adding a new row
  new_row_df = pd.DataFrame(curr_new_dict, index =[0])
  # Concatenate new_row with df
  updated_df = pd.concat([new_row_df, df[:]]).reset_index(drop = True)
  return updated_df

In [ ]:
def extract_exam_one_data(dfs):
  data_df = pd.DataFrame()
  previous_column = ""
  for idx in range(len(dfs)):
    # Step-1: drop unwanted columns
    raw_df = drop_unwanted_columns(dfs[idx])

    # Step-1: skip loop if columns does not exists
    if len(raw_df.columns) == 0:
      continue

    # Step-2: if it is "Test Name", "Diagnosis Code", "Doctor Specialty" and "Lab" table             
    if raw_df.columns[0] not in ["Response Status", "Condition"]:
      # Step-3: if it is first "Encounter" table
      if len(data_df) != 9 and raw_df.columns[0] == "Encounter":
        cols_list = ["Score", "Lab", "Doctor Name", "Location", "Phone"]
        for col_name in cols_list:
          raw_df[col_name] = ""

      # Step-4: handle table that dont have column header
      if raw_df.columns[0] not in ["Encounter", "Test Name", "Diagnosis Code", "Doctor Specialty", "Lab", "21", "0"]:
        previous_column = raw_df.columns[0]
        add_column_header(raw_df, previous_column)

      # Step-5: otherwise, assign column value to "Encounter" table's column based on index position
      updated_df = update_df_columns(raw_df)  # rename and add new column to df
      data_df = pd.concat([data_df, updated_df], ignore_index=True)

  return data_df

In [ ]:
data_df = extract_exam_one_data(dfs)

In [ ]:
len(data_df)

1221

In [ ]:
data_df.head()

,Encounter,Specimen,Service Date,High Score,Score,Lab,Doctor Name,Location,Phone
0,Encounter,Specimen,Service Date,High Score,Score,Lab,Doctor Name,Location,Phone
1,30,NaN,2021-09-24,5,,,,,
2,NaN,NaN,NaN,NaN,,,,,
3,Diagnosis Code,Description,Code Type,Score,,,,,
4,D649,"Anemia, unspecified",ICD10,5,,,,,


In [ ]:
data_df = data_df.drop([data_df.index[0]])
data_df.head()

,Encounter,Specimen,Service Date,High Score,Score,Lab,Doctor Name,Location,Phone
1,30,NaN,2021-09-24,5,,,,,
2,NaN,NaN,NaN,NaN,,,,,
3,Diagnosis Code,Description,Code Type,Score,,,,,
4,D649,"Anemia, unspecified",ICD10,5,,,,,
5,E611,Iron deficiency,ICD10,5,,,,,


In [ ]:
data_df.to_csv("exam-one.csv")